<a href="https://colab.research.google.com/github/Sidarth-V/UsedCarApp/blob/FormPage/Carpricenew.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd 
import numpy as np 
import json 
import pickle 
from sklearn.ensemble import RandomForestRegressor 
from sklearn.model_selection import train_test_split 
from sklearn.model_selection import ShuffleSplit 
from sklearn.model_selection import cross_val_score 
from sklearn.model_selection import GridSearchCV 
from warnings import simplefilter 

simplefilter(action='ignore', category=FutureWarning)

In [2]:
df1 = pd.read_csv('cardekho_cleaned.csv')
df1.head()

,id,car_name,brand,model,new_price,min_cost_price,max_cost_price,vehicle_age,km_driven,seller_type,fuel_type,transmission_type,mileage,engine,max_power,seats,selling_price
0,0,Maruti Alto,Maruti,Alto,NaN,NaN,NaN,9,120000,Individual,Petrol,Manual,19.70,796,46.30,5,120000
1,1,Hyundai Grand,Hyundai,Grand,New Car (On-Road Price) : Rs.7.11-7.48 Lakh*,711000.0,748000.0,5,20000,Individual,Petrol,Manual,18.90,1197,82.00,5,550000
2,2,Hyundai i20,Hyundai,i20,NaN,NaN,NaN,11,60000,Individual,Petrol,Manual,17.00,1197,80.00,5,215000
3,3,Maruti Alto,Maruti,Alto,NaN,NaN,NaN,9,37000,Individual,Petrol,Manual,20.92,998,67.10,5,226000
4,4,Ford Ecosport,Ford,Ecosport,New Car (On-Road Price) : Rs.10.14-13.79 Lakh*,1014000.0,1379000.0,6,30000,Dealer,Diesel,Manual,22.77,1498,98.59,5,570000


Among the data given, the columns corresponding to car name,seller type, brand and model have little relevance to car prices and given that these arent constant and other models/brands can exist, including these will harm the model efficacy. The columns on minimum and maximum cost price are incomplete for most data in the given dataset and as a result it doesnt make sense to limit the data to only the ones with complete information and thereby restrict amount of data we have.

In [3]:
df2 = df1.drop(['car_name','seller_type','brand','model','new_price','id','min_cost_price','max_cost_price'],axis=1).reindex(['transmission_type','fuel_type','vehicle_age','km_driven','mileage','engine','max_power','seats','selling_price'],axis=1)
df2.head()

,transmission_type,fuel_type,vehicle_age,km_driven,mileage,engine,max_power,seats,selling_price
0,Manual,Petrol,9,120000,19.70,796,46.30,5,120000
1,Manual,Petrol,5,20000,18.90,1197,82.00,5,550000
2,Manual,Petrol,11,60000,17.00,1197,80.00,5,215000
3,Manual,Petrol,9,37000,20.92,998,67.10,5,226000
4,Manual,Diesel,6,30000,22.77,1498,98.59,5,570000


In [4]:
df4 = df2
df4.fuel = df4.fuel_type.map(lambda x: x if x in ['Diesel','Petrol'] else 'Other')
df4.fuel.unique()
manual = df4[df4['transmission_type']=='Manual']
automatic = df4[df4['transmission_type']=='Automatic']

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  


In [5]:
df5=df4
km_mean = df5.groupby('vehicle_age')['km_driven'].mean()
year_mean_manual = df5[df5['transmission_type']=='Manual'].groupby('vehicle_age')['selling_price'].mean()
year_mean_automatic = df5[df5['transmission_type']=='Automatic'].groupby('vehicle_age')['selling_price'].mean()
df5.groupby('transmission_type').agg(['mean','std','min','max']).applymap(lambda x: format(x,'.0f')).drop(['vehicle_age'],axis=1)

km_driven               ... selling_price                 
                       mean    std   min  ...           std    min       max
transmission_type                         ...                               
Automatic             48813  40964  1000  ...       1667566  80000  39500000
Manual                60105  52399   100  ...        318334  30000   8100000

[2 rows x 24 columns]

At this point we can see the presence of very few/limited outliers in the data and trying to remove the outliers leads to loss in overall data with very little gain in accuracy considering the tradeoff of amount of data. As a result, we proceed with the outliers.

In [6]:
transmission = pd.get_dummies(df5['transmission_type'])
fuel = pd.get_dummies(df5['fuel_type'])
year=df5['vehicle_age']
mileage=df5['mileage']
engine=df5['engine']
max_power=df5['max_power']
seats=df5['seats']

X = pd.concat([transmission,fuel,df5.drop(['transmission_type','fuel_type','selling_price'],axis=1)],axis=1)
y = df5['selling_price']

In [7]:
X.head(5)

,Automatic,Manual,CNG,Diesel,Electric,LPG,Petrol,vehicle_age,km_driven,mileage,engine,max_power,seats
0,0,1,0,0,0,0,1,9,120000,19.70,796,46.30,5
1,0,1,0,0,0,0,1,5,20000,18.90,1197,82.00,5
2,0,1,0,0,0,0,1,11,60000,17.00,1197,80.00,5
3,0,1,0,0,0,0,1,9,37000,20.92,998,67.10,5
4,0,1,0,1,0,0,0,6,30000,22.77,1498,98.59,5


In [8]:
y.head(5)

0    120000
1    550000
2    215000
3    226000
4    570000
Name: selling_price, dtype: int64

In [9]:
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2,random_state=10)

To identify the best regression model, we use GridSearchCV using the following code:


 

```
for algo_name, config in algos.items():
 
        gs =  GridSearchCV(config['model'], config['params'], cv=cv, return_train_score=False)
        gs.fit(X,y)
        scores.append({
            'model': algo_name,
            'best_score': gs.best_score_,
            'best_params': gs.best_params_
        })
```


At this point, we obtain the best model as random forest speciically with mean square error and 26 trees in the forest. 

In [10]:
model = RandomForestRegressor(criterion='mse',n_estimators=26)

In [11]:
model.fit(X_train,y_train)

RandomForestRegressor(bootstrap=True, ccp_alpha=0.0, criterion='mse',
                      max_depth=None, max_features='auto', max_leaf_nodes=None,
                      max_samples=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=2, min_weight_fraction_leaf=0.0,
                      n_estimators=26, n_jobs=None, oob_score=False,
                      random_state=None, verbose=0, warm_start=False)

In [ ]:
from sklearn.linear_model import LinearRegression #Linear Regression
from sklearn.linear_model import Lasso #Lasso Regression
from sklearn.tree import DecisionTreeRegressor

def find_best_model_using_gridsearchcv(X,y):
    algos = {
        'linear_regression' : {
            'model': LinearRegression(),
            'params': {
                'normalize': [True, False]
            }
        },
        'lasso': {
            'model': Lasso(),
            'params': {
                'alpha': [1,2],
                'selection': ['random', 'cyclic']
            }
        },
        'decision_tree': {
            'model': DecisionTreeRegressor(),
            'params': {
                'criterion' : ['mse','friedman_mse'],
                'splitter': ['best','random']
            }
        },
        'random_forest': {
            'model': RandomForestRegressor(),
            'params': {
                'criterion' : ['mse','friedman_mse'],
                'n_estimators': range(1,50,5)
            }
        }
    }
    scores = []
    cv = ShuffleSplit(n_splits=5, test_size=0.2, random_state=0)
    for algo_name, config in algos.items():
        gs =  GridSearchCV(config['model'], config['params'], cv=cv, return_train_score=False)
        gs.fit(X,y)
        scores.append({
            'model': algo_name,
            'best_score': gs.best_score_,
            'best_params': gs.best_params_
        })

    return pd.DataFrame(scores,columns=['model','best_score','best_params'])

find_best_model_using_gridsearchcv(X,y)

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_coordinate_descent.py:476: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 2238967768303310.8, tolerance: 1234605993635.0554
  positive)
/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_coordinate_descent.py:476: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 2580829186271637.5, tolerance: 1368690422438.6833
  positive)
/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_coordinate_descent.py:476: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 2031586343256143.5, tolerance: 1180919929337.1697
  positive)
/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_coordinate_descent.py:476: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 250051

In [12]:
def predict_price(transmission,fuel,km_driven,year,mileage,engine,max_power,seats):
    x = []
    x[:12] = np.zeros(13,dtype='int32')
    x[7] = year
    x[8] = km_driven
    x[9] = mileage
    x[10] = engine
    x[11] = max_power
    x[12] = seats
    transmission_index = np.where(X.columns==transmission)[0][0]
    fuel_index = np.where(X.columns==fuel)[0][0]
    
    if transmission_index>=0:
        x[transmission_index] = 1
    if fuel_index>=2:
        x[fuel_index] = 1
        
    return float(format(model.predict([x])[0],'.2f'))

In [13]:
x=predict_price('Manual','Petrol',120000,9,19.70,796,46.30,5)
print('Range of price is :' , round(x-40000,2) , ' to ' , x)

Range of price is : 130692.31  to  170692.31
